# Packages & Preloaded Data

In [1]:
from lxml import etree
import lxml.html
import requests
import pandas as pd
from collections import ChainMap
from io import StringIO, BytesIO
import os
from functools import reduce
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## CIK Reference Data

Below is a list of URLs, maintained by the SEC, that provide CIK's for companies registered with the SEC.
* *ticker.txt* contains CIK and ticker
* *company_tickers.json* contains CIK, ticker and company name
* *cik-lookup-data.txt* contains full CIK history, structure is similar to *company_tickers.json* but with history


In [2]:
cik_urls = ['https://www.sec.gov/include/ticker.txt',
            'https://www.sec.gov/files/company_tickers.json',
           'https://www.sec.gov/Archives/edgar/cik-lookup-data.txt']

data_cik = pd.read_json(cik_urls[1]).T
display(data_cik.head())

,cik_str,ticker,title
0,1750,AIR,AAR CORP
1,1800,ABT,ABBOTT LABORATORIES
10,2488,AMD,ADVANCED MICRO DEVICES INC
100,8497,ASAM,ASSURANCEAMERICA CORP
1000,94344,STC,STEWART INFORMATION SERVICES CORP


# Get CIK for Ticker(s)

In [3]:
def get_cik(ticker):
    if type(ticker) == str:
        search_list = list()
        search_list.append(ticker)
    else:
        search_list = ticker
    df = data_cik[data_cik['ticker'].isin(search_list)].reset_index(drop = True)
    return(df)

In [4]:
tickers = ['WFC', 'JPM', 'CIT', 'HBAN']
banks = get_cik(tickers)
display(banks)

,cik_str,ticker,title
0,19617,JPM,JPMORGAN CHASE & CO
1,49196,HBAN,HUNTINGTON BANCSHARES INC/MD
2,72971,WFC,WELLS FARGO & COMPANY/MN
3,1171825,CIT,CIT GROUP INC


# Get List of Filings for CIK(s)

In [5]:
def get_filings(cik, filing_type, date_before = '', nrows = ''):
    if type(cik) == str:
        cik_list = list()
        cik_list.append(cik)
    else:
        cik_list = cik
        
    urls = []
    for cik in cik_list:
        url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=%s&type=%s&dateb=%s&count=%s&owner=exclude&output=atom' % (cik, filing_type, date_before, nrows)
        xml = requests.get(url, stream = True, verify = False)
        xml.raw.decode_content = True
        xml = etree.parse(xml.raw)
        urls.append(url)
    
    frame = []
    for url in urls:
        req = requests.get(url, stream = True, verify = False)
        req.raw.decode_content = True  # ensure transfer encoding is honoured
        tree = etree.parse(req.raw)
        
        cik = []
        for el in tree.iter('{*}cik'): cik.append(el.text)
        cik = cik[0]
        
        name = []
        for el in tree.iter('{*}conformed-name'): name.append(el.text)
        name = name[0]
        
        entries = []
        for el in tree.iter('{*}content'): entries.append(el)
        for entry in entries:
            elements = entry.getchildren()
            values = []
            for el in elements:
                values.append({el.tag : el.text})
            values = dict(ChainMap(*values))
            values.update({'CIK' : cik})
            values.update({'Name' : name})
            frame.append(values)
    
    frame = pd.DataFrame(frame)
    frame.columns = frame.columns.str.replace('{http://www.w3.org/2005/Atom}', '')
    frame['filing-date'] = pd.to_datetime(frame['filing-date'], infer_datetime_format = True)
    frame = frame[['CIK', 'Name', 'file-number-href', 'filing-type', 'xbrl_href',
       'form-name', 'act', 'film-number', 'file-number', 'filing-href', 'size',
       'filing-date', 'accession-nunber', 'amend']]
    return(frame)

We can get a list of 10-Ks for our defined universe of financial institutions & banks. We filter this set to the most recently reported 10-K for the calendar year of 2018.

In [6]:
bank_filings = get_filings(banks['cik_str'], '10-K')
bank_filings_recent = bank_filings[bank_filings['filing-date'] >= '1/1/2019'].reset_index(drop = True)
display(bank_filings_recent.head())

,CIK,Name,file-number-href,filing-type,xbrl_href,form-name,act,film-number,file-number,filing-href,size,filing-date,accession-nunber,amend
0,0000019617,JPMORGAN CHASE & CO,https://www.sec.gov/cgi-bin/browse-edgar?actio...,10-K,https://www.sec.gov/cgi-bin/viewer?action=view...,"Annual report [Section 13 and 15(d), not S-K I...",34,19634240,001-05805,https://www.sec.gov/Archives/edgar/data/19617/...,85 MB,2019-02-26,0000019617-19-000054,NaN
1,0000049196,HUNTINGTON BANCSHARES INC/MD,https://www.sec.gov/cgi-bin/browse-edgar?actio...,10-K,https://www.sec.gov/cgi-bin/viewer?action=view...,"Annual report [Section 13 and 15(d), not S-K I...",34,19611545,001-34073,https://www.sec.gov/Archives/edgar/data/49196/...,38 MB,2019-02-15,0000049196-19-000018,NaN
2,0000072971,WELLS FARGO & COMPANY/MN,https://www.sec.gov/cgi-bin/browse-edgar?actio...,10-K,https://www.sec.gov/cgi-bin/viewer?action=view...,"Annual report [Section 13 and 15(d), not S-K I...",34,19637386,001-02979,https://www.sec.gov/Archives/edgar/data/72971/...,95 MB,2019-02-27,0000072971-19-000227,NaN
3,0001171825,CIT GROUP INC,https://www.sec.gov/cgi-bin/browse-edgar?actio...,10-K,https://www.sec.gov/cgi-bin/viewer?action=view...,"Annual report [Section 13 and 15(d), not S-K I...",34,19622681,001-31369,https://www.sec.gov/Archives/edgar/data/117182...,60 MB,2019-02-21,0001564590-19-003711,NaN


# Get List of XBRL Files for given Filing(s) & Basic XBRL Parsing

In [7]:
# Given a pd.DataFrame of filings, returns a tidy pd.DataFrame containing information on the XBRL documents associated with filings
def get_filings_xbrl_files(filings):
    urls = filings.values.tolist()
    filings = pd.DataFrame()
    for url in urls:
        base = os.path.dirname(url[9])

        r = requests.get(url[9], verify = False)
        tree = lxml.html.fromstring(r.text)
        table = tree.xpath('//table')[1]
        f = pd.read_html(lxml.html.tostring(table, method = 'html', encoding = 'unicode'))[0]
        f.columns = f.iloc[0,:]
        f = f.iloc[1:,:]
        f.loc[:,'CIK'] = url[0]
        f.loc[:,'Firm'] = url[1]
        f.loc[:,'Date'] = url[11]
        f.loc[:,'URL'] = base + r'/' + f.loc[:,'Document']
        filings = pd.concat([filings, f])
    filings.reset_index(inplace = True, drop = True)
    return(filings)

# Returns a list of all tags available within an XBRL file.
def get_tag_list(url):
    data_url = url
    data_request = requests.get(data_url, stream = False, verify = False)
    data_tree = etree.ElementTree(etree.XML(data_request.content, parser = etree.XMLParser(encoding = "US-ASCII")))

    data_tags = set()
    for element in data_tree.iter():
        data_tags.add(element.tag)
    return(data_tags)

# Returns the value(s), in a pd.DataFrame, of a specified tag (line_item) within a given context [axis?]
def get_tag_value(url, line_item, context_ref):
    data_url = url
    data_request = requests.get(data_url, stream = False, verify = False)
    data_tree = etree.ElementTree(etree.XML(data_request.content, parser = etree.XMLParser(encoding = "US-ASCII")))

    data_tags = set()
    for element in data_tree.iter():
        data_tags.add(element.tag)

    search = '//{*}' + line_item #SEARCH PARAM HERE, such as NetIncomeLoss
    response = []
    for member in data_tree.findall(search):
        # print(member.tag, member.text, member.attrib['contextRef'])
        response.append([member.attrib['contextRef'], member.text, member.tag])
    try:
        response = pd.DataFrame(sorted(response)).drop_duplicates()
        response = response[response[0].str.contains(context_ref, regex = True)] #'FD.*?Q4YTD$'
        return(response)
    except:
        return(pd.DataFrame())

In [8]:
bank_filings_files = get_filings_xbrl_files(bank_filings_recent)
display(bank_filings_files[(bank_filings_files['Type'] == 'EX-101.INS') | (bank_filings_files['Type'] == 'XML')])

,Seq,Description,Document,Type,Size,CIK,Firm,Date,URL
5,25,EXTRACTED XBRL INSTANCE DOCUMENT,corp10k2018_htm.xml,XML,24262678,0000019617,JPMORGAN CHASE & CO,2019-02-26,https://www.sec.gov/Archives/edgar/data/19617/...
6,10,XBRL INSTANCE DOCUMENT,hban-20181231.xml,EX-101.INS,10733132,0000049196,HUNTINGTON BANCSHARES INC/MD,2019-02-15,https://www.sec.gov/Archives/edgar/data/49196/...
12,16,XBRL INSTANCE DOCUMENT,wfc-20181231.xml,EX-101.INS,32732284,0000072971,WELLS FARGO & COMPANY/MN,2019-02-27,https://www.sec.gov/Archives/edgar/data/72971/...
18,12,XBRL INSTANCE DOCUMENT,cit-20181231.xml,EX-101.INS,17633267,0001171825,CIT GROUP INC,2019-02-21,https://www.sec.gov/Archives/edgar/data/117182...


In [9]:
for url in bank_filings_files[(bank_filings_files['Type'] == 'EX-101.INS') | (bank_filings_files['Type'] == 'XML')]['URL']:
    print(bank_filings_files[bank_filings_files.loc[:,'URL'] == url].iloc[0,6])
    display(get_tag_value(url, 'Assets', 'FI.*?Q4$'))

JPMORGAN CHASE & CO


,0,1,2
0,FI2016Q4,2490972000000,{http://fasb.org/us-gaap/2018-01-31}Assets
13,FI2017Q4,2533600000000,{http://fasb.org/us-gaap/2018-01-31}Assets
38,FI2018Q4,2622532000000,{http://fasb.org/us-gaap/2018-01-31}Assets


HUNTINGTON BANCSHARES INC/MD


,0,1,2
0,FI2017Q4,104185000000,{http://fasb.org/us-gaap/2018-01-31}Assets
7,FI2018Q4,108781000000,{http://fasb.org/us-gaap/2018-01-31}Assets


WELLS FARGO & COMPANY/MN


,0,1,2
0,FI2017Q4,1951757000000,{http://fasb.org/us-gaap/2018-01-31}Assets
18,FI2018Q4,1895883000000,{http://fasb.org/us-gaap/2018-01-31}Assets


CIT GROUP INC


,0,1,2


In [10]:
display(get_tag_value(bank_filings_files.iloc[5,8], 'TierOneRiskBasedCapital', '.*?ParentCompanyMember.*?StandardizedTransitionalMember'))

,0,1,2
5,FI2017Q4_srt_ConsolidatedEntitiesAxis_srt_Pare...,208644000000,{http://fasb.org/us-gaap/2018-01-31}TierOneRis...
11,FI2018Q4_srt_ConsolidatedEntitiesAxis_srt_Pare...,209093000000,{http://fasb.org/us-gaap/2018-01-31}TierOneRis...


In [11]:
get_tag_list(bank_filings_files.iloc[5,8])

{'{http://www.jpmorganchase.com/20181231}OffBalanceSheetLendingRelatedFinancialCommitmentsContractualAmountEvaluatedAtFormulaBasedImpairmentMethodology',
 '{http://fasb.org/us-gaap/2018-01-31}IncomeTaxExpenseBenefit',
 '{http://www.jpmorganchase.com/20181231}DefinedBenefitPlanPercentAboveWhichAmortizationOfNetGainsAndLossesIsIncludedInAnnualNetPeriodicBenefitCost',
 '{http://fasb.org/us-gaap/2018-01-31}SharebasedCompensationArrangementBySharebasedPaymentAwardAwardVestingRightsPercentage',
 '{http://fasb.org/us-gaap/2018-01-31}DebtSecuritiesTradingMeasurementInput',
 '{http://fasb.org/us-gaap/2018-01-31}ScheduleOfSegmentReportingInformationBySegmentTextBlock',
 '{http://www.jpmorganchase.com/20181231}InvestmentSecuritiesContinuousUnrealizedLossPositionLessthan12MonthsAggregateLossPointinTime',
 '{http://fasb.org/us-gaap/2018-01-31}OtherSignificantNoncashTransactionValueOfConsiderationGiven1',
 '{http://www.jpmorganchase.com/20181231}SecuritiesTransferredToPrivateLabelResecuritizationVie

In [12]:
tags = []
for url in bank_filings_files[(bank_filings_files['Type'] == 'EX-101.INS') | (bank_filings_files['Type'] == 'XML')]['URL']:
    t = get_tag_list(url)
    tags.append(t)
len(tags)

4

In [13]:
x = list(reduce(lambda i, j: i & j, (set(x) for x in tags)))
print("Length:",len(x))
x

Length: 207


['{http://fasb.org/us-gaap/2018-01-31}OtherComprehensiveIncomeLossAmortizationAdjustmentFromAOCIPensionAndOtherPostretirementBenefitPlansForNetPriorServiceCostCreditBeforeTax',
 '{http://fasb.org/us-gaap/2018-01-31}IncomeTaxExpenseBenefit',
 '{http://fasb.org/us-gaap/2018-01-31}ScheduleOfSegmentReportingInformationBySegmentTextBlock',
 '{http://www.xbrl.org/2003/instance}measure',
 '{http://fasb.org/us-gaap/2018-01-31}ShareBasedCompensationArrangementByShareBasedPaymentAwardEquityInstrumentsOtherThanOptionsNonvestedNumber',
 '{http://xbrl.sec.gov/dei/2018-01-31}AmendmentFlag',
 '{http://fasb.org/us-gaap/2018-01-31}ShareBasedCompensationArrangementByShareBasedPaymentAwardEquityInstrumentsOtherThanOptionsGrantsInPeriodWeightedAverageGrantDateFairValue',
 '{http://fasb.org/us-gaap/2018-01-31}ShareBasedCompensationArrangementByShareBasedPaymentAwardEquityInstrumentsOtherThanOptionsForfeitedInPeriod',
 '{http://fasb.org/us-gaap/2018-01-31}DefinedBenefitPlanAmortizationOfGainsLosses',
 '{htt